#### Note: This will run if you have the the movies saved as "start_numor".npz files. Check other notebook titled SANS_to_npz.ipynb for code to assist in doing this.

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
from scipy.signal import savgol_filter
#from bg_mpl_stylesheets.styles import all_styles
from scipy.ndimage import gaussian_filter

#plt.style.use(all_styles["bg_style"])

In [ ]:
# Mask for images

def mask_and_blur_images(movie, sigma=.65):
    '''masks signal inside radius of 14 and outside radius of 30 and adds gaussian blur for all intensity data'''
    #print(f"movie type: {type(movie)}, movie value: {movie}")
    for i in range(0, movie.shape[0]):
        x,y = np.meshgrid(np.arange(128), np.arange(128))
        radius = np.sqrt((x-64)**2 + (y-62)**2)
        mask1 = radius <= 14
        mask2 = radius >= 30
        masked_data = movie[i].copy()
        masked_data[mask1] = 0
        masked_data2 = masked_data.copy()
        masked_data2[mask2] = 0
        # masked_data_norm = (masked_data - np.min(masked_data) / (np.max(masked_data) - np.min(masked_data)))
        blurred_data = gaussian_filter(masked_data2, sigma=sigma)
        movie[i] = blurred_data
    return movie

In [ ]:
# import images from .npz files
ms = torch.arange(12)
angles_rad = torch.arange(0, 6) * 2 * torch.pi / 6.

# Extract data_theta, doesn't matter what images is extracted since we're just getting theta
data = np.load(r"C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Angle reference file (random file from Caden)\image_111010.npz")['data']
data_theta = torch.atan2(torch.tensor(data[1]), torch.tensor(data[0]))

# Extract data file paths
file_path = r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\HDF to npz files\\'
movies = ['Field_29mT.npz', 'Field_31mT.npz', 'Field_32mT.npz', 'Field_33mT.npz', 'Field_37mT.npz']

# define the movie you want to run GD and GS on as gif
gif = movies[0]
print(gif)
movie = np.load(file_path + gif)

#shape (<movie length>,128,128)
intensity_data = movie['data']
print(intensity_data.shape[0])
intensity_data = torch.tensor(mask_and_blur_images(intensity_data))
# Apply a log transformation to the intensity data
# Add a small constant (e.g., 1e-6) to avoid taking log(0)
#intensity_data = torch.log(intensity_data + 1e-6)
print(intensity_data.numpy().shape)
initial_offset = torch.tensor(0., requires_grad=True)
# plt.imshow(intensity_data[0])
# plt.show()
# plt.imshow(mask_and_blur_images(intensity_data.numpy())[0])

In [ ]:
# Normalize data

def normalize_min_max(array):
    array_min = np.min(array)
    array_max = np.max(array)
    return (array - array_min) / (array_max - array_min)

In [ ]:
# Create the filter

def project_theta(theta, m_values):
    projections = []
    for m in m_values:
        sin_m_theta = torch.sin(m * theta )
        cos_m_theta = torch.cos(m * theta )
        projected_vectors = torch.stack((cos_m_theta, sin_m_theta), axis=-1)
        projections.append(projected_vectors)
    return torch.stack(projections, axis=0)

def evaluate_functions_on_theta(theta, coefficients_list, m_values):
    evaluated_function = torch.zeros(theta.shape, dtype=torch.float32)
    for (a_cos, a_sin), m in zip(coefficients_list, m_values):
        evaluated_function += a_sin * torch.sin(m * theta) + a_cos * torch.cos(m * theta)
    return evaluated_function

In [ ]:
# gradient descent function

def optimize_offset(intensity, offset):
    max_iter = 101
    opt = torch.optim.Adam([offset], lr=1e-2)
    for i in range(max_iter):
        projection = project_theta(angles_rad + offset, ms).sum(1)
        evaluate_image_theta = evaluate_functions_on_theta(data_theta, projection, ms)
        loss = -(intensity * evaluate_image_theta).sum()
        opt.zero_grad()
        loss.backward()
        opt.step()
        #if i % 100 == 0:
        #    print(loss.item(), offset.item())
    return offset, evaluate_image_theta, intensity

## Gradient Descent

In [ ]:
# gif gradient descent (for comparison with grid search function)
gradient_descent_offset = []
initial_offset = torch.tensor(0., requires_grad=True)
print('data for movie', gif)
for i in range(intensity_data.shape[0]):
    offset, image, int = optimize_offset(intensity_data[i], initial_offset)
    gradient_descent_offset.append(np.rad2deg(offset.item()))
    signal = normalize_min_max(image.detach().numpy()) + normalize_min_max(int.detach().numpy())
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(signal, cmap='jet', vmin=0, vmax=2)
    axs[1].imshow(int, cmap='jet')
    plt.show()
    print(f'{i*10}s:', np.rad2deg(offset.item()))

## Grid Search

In [ ]:
def grid_search_offset(movie, offset_step=.2, angle_below_offset=8, angle_above_offset=3):
    # offset_step_rad = np.deg2rad(offset_step)
    optimal_offset = []

    for index, image in enumerate(movie):
        loss_offset = []
        if index == 0:
            offset_angles_deg = np.arange(-45, 45, offset_step)
        else:
            offset_angles_deg = np.arange(best_offset-angle_below_offset, best_offset+angle_above_offset, offset_step)

        for offset_angle in offset_angles_deg:
            offset = torch.tensor([offset_angle], dtype=torch.float32)
            projection = project_theta(angles_rad + np.deg2rad(offset), ms).sum(1)
            evaluate_image_theta = evaluate_functions_on_theta(data_theta, projection, ms)
            image = image.numpy()
            image = normalize_min_max(image)
            image = torch.tensor(image)
            loss = -(image * evaluate_image_theta).sum()
            loss_offset.append((offset_angle, loss))
        min_loss = min(loss_offset, key=lambda x: x[1])
        best_offset = min_loss[0]

        # plotting
        norm_int = normalize_min_max(image.numpy())
        norm_filter = normalize_min_max(evaluate_image_theta.numpy())
        signal = norm_int + norm_filter
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        axs[0].imshow(signal, cmap='jet', vmin=0, vmax=2)
        axs[1].imshow(norm_int, cmap='jet')
        plt.show()
        optimal_offset.append(best_offset)
        print(f'{index*10}s: ', best_offset)

    return optimal_offset

ratchet_offset = grid_search_offset(intensity_data)

In [ ]:
# plot data created from above GD and GS

time = np.array(range(len(ratchet_offset))) * 10
#print(gradient_descent_offset[40])
print(ratchet_offset[0])
#plt.plot(time, ratchet_offset, label='ratchet model', color='blue', alpha=.7)
plt.plot(time, gradient_descent_offset, label='gradient descent', color='red', alpha=.7)
plt.xlabel('time (s)')
plt.ylabel('offset angle (deg)')
#plt.title(gif.removesuffix('.npz'))
plt.grid(True)
plt.legend()


## Save Data to .npz

In [ ]:
# save data and figure
plt.plot(time, ratchet_offset, label='ratchet model', color='blue', alpha=.7)
#plt.plot(time, gradient_descent_offset, label='gradient descent', color='red', alpha=.7)
plt.xlabel('time (s)')
plt.ylabel('offset angle (deg)')
plt.title(gif)
plt.grid(True)
plt.legend()

# Save the plot
# plt.savefig(f'/Users/cadenmyers/billingelab/dev/skyrmion_lattices/comparing_models/plots/{gif.removesuffix('.npz')}_plot.png')

# Save the gradient descent data
#gradient_descent_offset=np.array(gradient_descent_offset)
# np.savez('/Users/cadenmyers/billingelab/dev/skyrmion_lattices/comparing_models/gradient_offset/gradient_descent_'+gif, offset=gradient_descent_offset, time=time)

# Save the ratchet model data
ratchet_offset=np.array(ratchet_offset)
# Define your file path
file_path = r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\\'

# Concatenate the file path with the filename stored in gif
full_path = file_path + gif

# Save the npz file using the full path
np.savez(full_path, gif, offset=ratchet_offset, time=time)

# Plot data from saved npz files
### everything below this is data analysis which includes plotting offset angle vs. time and calculating gradient

In [ ]:
# # trim cooldown image data (eliminate irrelevant signal)
# cooldown_ratchet_model_data = np.load('/Users/cadenmyers/billingelab/dev/skyrmion_lattices/comparing_models/ratchet_offset/ratchet_model_121405.npz')
# rm_time = cooldown_ratchet_model_data['time']
# rm_offset = cooldown_ratchet_model_data['offset']

# # Define the threshold
# threshold = 2000

# # Filter the data
# mask = rm_time >= threshold
# filtered_time = rm_time[mask]-2000
# filtered_offset = rm_offset[mask]
# filtered_offset = filtered_offset-filtered_offset[0]

In [ ]:
# save data
# np.savez('/Users/cadenmyers/billingelab/dev/skyrmion_lattices/comparing_models/ratchet_offset/ratchet_model_121405.npz',
#          time=filtered_time, offset=filtered_offset)
# # print(filtered_time)
# # print(filtered_offset)

# cooldown_ratchet_model_data = np.load('/Users/cadenmyers/billingelab/dev/skyrmion_lattices/comparing_models/ratchet_offset/ratchet_model_121405.npz')
# rm_time = cooldown_ratchet_model_data['time']
# rm_offset = cooldown_ratchet_model_data['offset']

# print(rm_time.shape)
# print(rm_offset.shape)

In [ ]:
# plot data from .npz files

#for gif in movies:
#gradient_descent_data = np.load(rf'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\npz files\testing\{gif}')
#gd_time = gradient_descent_data['time']
#gd_offset = gradient_descent_data['offset']
#plt.plot(gd_time, gd_offset, label=gif.removesuffix('.npz'), alpha=.7)

#plt.xlabel('time (s)')
#plt.xlim(0,600)
#plt.ylabel('offset angle (deg)')
#plt.title('gradient descent')
#plt.grid(True)
#plt.legend(loc='upper right', bbox_to_anchor=(1.35,1))

#plt.savefig('/Users/cadenmyers/billingelab/dev/skyrmion_lattices/comparing_models/plots/gradient_descent_offset_plot.png')
#plt.show()

for gif in movies:
    ratchet_model_data = np.load(rf'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\{gif}')
    rm_time = ratchet_model_data['time']
    rm_offset = ratchet_model_data['offset']
    m, b = np.polyfit(rm_time, rm_offset, 1)
    best_fit = m * rm_time + b
    SS_tot = np.sum((rm_offset - np.mean(rm_offset))**2)
    SS_res = np.sum((rm_offset - best_fit)**2)
    R_squared = 1 - (SS_res / SS_tot)
    range = np.abs(np.min(rm_offset)-np.max(rm_offset))
    # print(f'{gif} R^2=', R_squared)
    # print(f'{gif} range=', range)
    if R_squared >= .95 or range<=8:
        #plt.plot(rm_time, best_fit, label=gif.removesuffix('.npz'), alpha=.7)
        plt.plot(rm_time, best_fit, label=gif, alpha=.7)
    else:
        #plt.plot(rm_time, rm_offset, label=gif.removesuffix('.npz'), alpha=.7)
        plt.plot(rm_time, rm_offset, label=gif, alpha=.7)

plt.xlabel('time (s)')
plt.xlim(0,380)
plt.ylabel('offset angle (deg)')
plt.title('Ratchet Model')
plt.grid(True)
#plt.legend(loc='upper right', bbox_to_anchor=(1.35,1))
plt.legend(loc='center right')
plt.savefig(r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Exported Figures\FieldSweepPositions.png')
plt.show()

## Angular Velocity calculation

In [ ]:
def compute_smoothed_derivative(time, offset, window_length=11, polyorder=2):
    '''compute velocity of data after savgol_filter is applied'''
    smoothed_angle = savgol_filter(offset, window_length=window_length, polyorder=polyorder)
    time = np.array(time)
    smoothed_derivative = (np.gradient(smoothed_angle, time))
    return smoothed_derivative

In [ ]:
# calculate angular velo and plot
for gif in movies:
    ratchet_model_data = np.load(rf'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\{gif}')
    rm_time = ratchet_model_data['time']
    rm_offset = ratchet_model_data['offset']
    m, b = np.polyfit(rm_time, rm_offset, 1)
    best_fit = m * rm_time + b
    SS_tot = np.sum((rm_offset - np.mean(rm_offset))**2)
    SS_res = np.sum((rm_offset - best_fit)**2)
    R_squared = 1 - (SS_res / SS_tot)
    range = np.abs(np.min(rm_offset)-np.max(rm_offset))
    #print(f'{gif.removesuffix('.npz')} R^2=', R_squared)
    print(f'{gif} R^2=', R_squared)
    # print(f'{gif} range=', range)
    if R_squared >= .98 or range<=8:
        velo = compute_smoothed_derivative(rm_time, best_fit)
        #print(f'{gif.removesuffix('.npz')} avg angular velo=', m, 'deg/s')
        print(f'{gif} avg angular velo=', m, 'deg/s')
    else:
        velo = compute_smoothed_derivative(rm_time, rm_offset)

    #plt.plot(rm_time, velo, label=gif.removesuffix('.npz'), alpha=.7)
    plt.plot(rm_time, velo, label=gif, alpha=.7)

plt.xlabel('time (s)')
plt.xlim(0,380)
plt.ylabel('Angular Velocity (deg/s)')
plt.title('Ratchet Model')
plt.grid(True)
plt.legend(loc='upper right', bbox_to_anchor=(1.35,1))
plt.savefig(r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Exported Figures\FieldSweepVelocities.png')
plt.show()

# Gradient Descent (2 filters)

In [ ]:
import numpy as np
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt
import torch

In [ ]:
########## Create filter
def project_theta(theta, m_values):
    projections = []
    for m in m_values:
        sin_m_theta = torch.sin(m * theta)
        cos_m_theta = torch.cos(m * theta)
        projected_vectors = torch.stack((cos_m_theta, sin_m_theta), axis=-1)
        projections.append(projected_vectors)
    return torch.stack(projections, axis=0)

def evaluate_functions_on_theta(theta, coefficients_list, m_values):
    evaluated_function = torch.zeros(theta.shape, dtype=torch.float32)    
    for (a_cos, a_sin), m in zip(coefficients_list, m_values):
        evaluated_function += a_sin * torch.sin(m * theta) + a_cos * torch.cos(m * theta)
    return evaluated_function

########## Create masks
def normalize_image(image):
    return (image - image.min()) / (image.max() - image.min())

def apply_threshold_to_sin(evaluate_image, threshold=-1):
    """Thin out the laser by creating a mask from `evaluate_image`, setting values below `threshold` to 0.
    When `threshold = -1` this function doesn't change anything, when `threshold = 1` the image is completely white, 
    so typically threshold is between 0 and 1."""
    mask = torch.ones_like(evaluate_image, dtype=torch.float32)
    evaluate_image = normalize_image(evaluate_image)
    mask[evaluate_image < threshold] = 0
    masked_image = evaluate_image * mask
    return masked_image

def create_mask_from_intensity(intensity, evaluate_image):
    """Mask regions in `evaluate_image` where the values are positive, setting these regions in `intensity` to 0.
    This allows for a masked intensity image to be used in multiple filterings."""
    mask = torch.ones_like(evaluate_image, dtype=torch.float32)
    mask[evaluate_image > 0] = 0
    masked_intensity = intensity * mask
    return masked_intensity

In [ ]:
# Gradient Descent with 2 filters
# threshold = -1 means no change to the laser
# fix_snapback = True when you want to set a range to avoid snapback
# initial tells if this is the first frame

def optimize_offset_2filters(intensity, offset1, threshold=-1,
                             initial=True, fix_snapback=False, angle_below_offset=51, angle_above_offset=9):

    ########## start of the first filter
    # Define parameters
    max_iter_offset1 = 101
    lr1 = 1e-2
    opt1 = torch.optim.Adam([offset1], lr=lr1)

    # Set offset1 range based on whether it's the first frame
    offset1_range = (
        (-30 / 180 * np.pi, 30 / 180 * np.pi) if initial 
        else (offset1 - angle_below_offset / 180 * np.pi, offset1 + angle_above_offset / 180 * np.pi)
    )
    
    for i in range(max_iter_offset1):
        projection1 = project_theta(angles + offset1, ms).sum(1)
        evaluate_image_theta1 = evaluate_functions_on_theta(data_theta, projection1, ms)
        evaluate_image_theta1 = apply_threshold_to_sin(evaluate_image_theta1, threshold=threshold) # thin out laser if needed
        loss1 = -(intensity * evaluate_image_theta1).sum()
        opt1.zero_grad()
        loss1.backward()
        opt1.step()

    # Ensure offset1 stays within bounds if needed
    adjustment = 60 / 180 * np.pi
    if fix_snapback:
        with torch.no_grad():
            while not (offset1_range[0] <= offset1 <= offset1_range[1]):
                offset1 += adjustment if offset1 < offset1_range[0] else -adjustment
    ########## end of the first filter

    ########## start of the second filter
    offset2 = torch.tensor(0., requires_grad=True)
    max_iter_offset2 = 101
    lr2 = 1e-2
    opt2 = torch.optim.Adam([offset2], lr=lr2)
    masked_intensity = create_mask_from_intensity(intensity, evaluate_image_theta1) # mask out intensity from the first laser
    for i in range(max_iter_offset2):
        # Second filter on masked intensity
        projection2 = project_theta(angles + offset2, ms).sum(1)
        evaluate_image_theta2 = evaluate_functions_on_theta(data_theta, projection2, ms)
        evaluate_image_theta2 = apply_threshold_to_sin(evaluate_image_theta2, threshold=threshold) # thin out laser if needed   
        loss2 = -(masked_intensity * evaluate_image_theta2).sum()
        opt2.zero_grad()
        loss2.backward()
        opt2.step()

    # If no strong signal from masked intensity data then we consider there're no splitting peaks and take offset2 = offset1
    # we can discuss more about how to set the condition, I'm not sure if this is the best way
    if masked_intensity.max() <= 0.6 * intensity.max():
        offset2 = offset1
        evaluate_image_theta2 = evaluate_image_theta1
    ########## end of the second filter

    # Plot with corrected position
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    ax[0].imshow(((evaluate_image_theta1 / evaluate_image_theta1.abs().max() + intensity / intensity.abs().max()).detach()).T, origin='lower')
    ax[1].imshow(((evaluate_image_theta2 / evaluate_image_theta2.abs().max() + intensity / intensity.abs().max()).detach()).T, origin='lower')
    plt.show()
    # print(loss1, loss2)

    return offset1, offset2


In [ ]:
# Define parameters
# Assume you have got the .npz files
ms = torch.arange(12)
angles = torch.arange(0, 6) * 2 * torch.pi / 6.

# Import images from .npz files
# Extract data_theta, doesn't matter what images is extracted since we're just getting theta
# data = np.load(r"C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Angle reference file (random file from Caden)\image_111010.npz")['data']
data = np.load('images/image_111019.npz')['data']
data_theta = torch.atan2(torch.tensor(data[1]), torch.tensor(data[0]))

# Extract data file paths
# file_path = r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\HDF to npz files\\'
file_path = "/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/Field_Sweep_data/"
movies = ['Field_29mT.npz', 'Field_31mT.npz', 'Field_32mT.npz', 'Field_33mT.npz', 'Field_37mT.npz']

# Define the movie you want to run GD and GS on as gif
gif = movies[4]
print(gif)
movie = np.load(file_path + gif)
intensity_data = torch.tensor(movie['data'])
print(intensity_data.shape)


In [ ]:
# Define parameters
offset_list1, offset_list2 = [], []
offset1 = torch.tensor(0.)
offset1.requires_grad = True
initial = True

# Loop through the movie
for index, image in enumerate(intensity_data):
    offset1, offset2 = optimize_offset_2filters(image, offset1, threshold=0.8, initial=initial)
    print(f'{(index + 1) * 10}s: offset 1 = {np.rad2deg(offset1.item())}, offset 2 = {np.rad2deg(offset2.item())}')
    initial = False
    offset_list1.append(np.rad2deg(offset1.item())), offset_list2.append(np.rad2deg(offset2.item()))

# Plot offset angles
time = np.array(range(len(offset_list1))) * 10 + 10
print(offset_list1)
plt.plot(offset_list1, label="offset1")
plt.legend()
plt.show()
plt.plot(offset_list1, label="offset1")
plt.plot(offset_list2, label="offset2")
plt.legend()
plt.show()

# Save model data
offset_list1=np.array(offset_list1)
# file_path = r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\\'
file_path = rf'/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/Field_Sweep_data/angles/'
full_path = file_path + gif
np.savez(full_path, gif, offset=offset_list1, time=time)



## Angular velocity calculation

In [ ]:
# Codes are directly from above
import numpy as np
import matplotlib.pyplot as plt

movies = ['Field_29mT.npz', 'Field_31mT.npz', 'Field_31mT_fix_snapback.npz', 'Field_32mT.npz', 'Field_33mT.npz', 'Field_37mT.npz']

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

for gif in movies:
    # ratchet_model_data = np.load(rf'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\{gif}')
    ratchet_model_data = np.load(rf'/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/Field_Sweep_data/angles/{gif}')
    rm_time = ratchet_model_data['time']
    rm_offset = ratchet_model_data['offset']
    
    m, b = np.polyfit(rm_time, rm_offset, 1)
    best_fit = m * rm_time + b
    SS_tot = np.sum((rm_offset - np.mean(rm_offset))**2)
    SS_res = np.sum((rm_offset - best_fit)**2)
    R_squared = 1 - (SS_res / SS_tot)
    range = np.abs(np.min(rm_offset)-np.max(rm_offset))
    if R_squared >= .95 or range<=8:
        ax[0].plot(rm_time, best_fit, label=gif + " (best fit)", alpha=.7)
    else:
        ax[0].plot(rm_time, rm_offset, label=gif, alpha=.7)
    ax[1].plot(rm_time, rm_offset, label=gif, alpha=.7)

# Formatting the first subplot (best fit)
ax[0].set_xlabel('time (s)')
ax[0].set_xlim(0, 380)
ax[0].set_ylabel('offset angle (deg)')
ax[0].grid(True)
ax[0].legend(loc='lower left', fontsize=9)
ax[0].set_title('Best Fit')

# Formatting the second subplot (original data)
ax[1].set_xlabel('time (s)')
ax[1].set_xlim(0, 380)
ax[1].set_ylabel('offset angle (deg)')
ax[1].grid(True)
ax[1].legend(loc='lower left', fontsize=9)
ax[1].set_title('Original Data')

# Adjust layout and show the plot
plt.tight_layout()
# plt.savefig(r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Exported Figures\FieldSweepPositions.png')
plt.savefig('angles.png')
plt.show()


In [ ]:
from scipy.signal import savgol_filter

def compute_smoothed_derivative(time, offset, window_length=11, polyorder=2):
    '''compute velocity of data after savgol_filter is applied'''
    smoothed_angle = savgol_filter(offset, window_length=window_length, polyorder=polyorder)
    time = np.array(time)
    smoothed_derivative = (np.gradient(smoothed_angle, time))
    return smoothed_derivative

In [ ]:
# calculate angular velo and plot
for gif in movies:
    # ratchet_model_data = np.load(rf'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\{gif}')
    ratchet_model_data = np.load(rf'/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/Field_Sweep_data/angles/{gif}')
    rm_time = ratchet_model_data['time']
    rm_offset = ratchet_model_data['offset']
    m, b = np.polyfit(rm_time, rm_offset, 1)
    best_fit = m * rm_time + b
    SS_tot = np.sum((rm_offset - np.mean(rm_offset))**2)
    SS_res = np.sum((rm_offset - best_fit)**2)
    R_squared = 1 - (SS_res / SS_tot)
    range = np.abs(np.min(rm_offset)-np.max(rm_offset))
    #print(f'{gif.removesuffix('.npz')} R^2=', R_squared)
    print(f'{gif} R^2=', R_squared)
    # print(f'{gif} range=', range)
    if R_squared >= .98 or range<=8:
        velo = compute_smoothed_derivative(rm_time, best_fit)
        #print(f'{gif.removesuffix('.npz')} avg angular velo=', m, 'deg/s')
        print(f'{gif} avg angular velo=', m, 'deg/s')
    else:
        velo = compute_smoothed_derivative(rm_time, rm_offset)

    #plt.plot(rm_time, velo, label=gif.removesuffix('.npz'), alpha=.7)
    plt.plot(rm_time, velo, label=gif, alpha=.7)

plt.xlabel('time (s)')
plt.xlim(0,380)
plt.ylabel('Angular Velocity (deg/s)')
# plt.title('Ratchet Model')
plt.grid(True)
plt.legend(loc='lower left')
# plt.savefig(r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Exported Figures\FieldSweepVelocities.png')
plt.savefig('angular_velocity.png')
plt.show()

## Temp Sweep Movies

In [ ]:
# Check if this model works on previous movies

from scipy.ndimage import gaussian_filter
def mask_and_blur_images(array):
    '''masks signal inside radius of 14 and outside radius of 30 and adds gaussian blur for all intensity data'''
    for i in range(0,60):
        x,y = np.meshgrid(np.arange(128), np.arange(128))
        radius = np.sqrt((x-64)**2 + (y-62)**2)
        mask1 = radius <= 14
        mask2 = radius >= 30
        masked_data = array[i].copy()
        masked_data[mask1] = 0
        masked_data2 = masked_data.copy()
        masked_data2[mask2] = 0
        # masked_data_norm = (masked_data - np.min(masked_data) / (np.max(masked_data) - np.min(masked_data)))
        blurred_data = gaussian_filter(masked_data2, sigma=.65)
        array[i] = blurred_data
    return array

# Define parameters
# Assume you have got the .npz files
ms = torch.arange(12)
angles = torch.arange(0, 6) * 2 * torch.pi / 6.

# Import images from .npz files
# Extract data_theta, doesn't matter what images is extracted since we're just getting theta
# data = np.load(r"C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Angle reference file (random file from Caden)\image_111010.npz")['data']
data = np.load('images/image_111019.npz')['data']
data_theta = torch.atan2(torch.tensor(data[1]), torch.tensor(data[0]))

# Extract data file paths
# file_path = r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\HDF to npz files\\'
file_path = "/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/Temp_Sweep_data/"
movies = ['118923.npz', '119486.npz', '119996.npz', '120506.npz', '121016.npz', '121405.npz', '121855.npz', '122365.npz', '122875.npz']

# Define the movie you want to run GD and GS on as gif
gif = movies[2]
print(gif)
movie = np.load(file_path + gif)
intensity_data = torch.tensor(mask_and_blur_images(movie['intensity']))
print(intensity_data.shape)


In [ ]:
# Define parameters
offset_list1, offset_list2 = [], []
offset1 = torch.tensor(0.)
offset1.requires_grad = True
initial = True

# Loop through the movie
# 119486.npz: angle_below_offset=57, angle_above_offset=3
# 121016.npz: 50, 10
# 121405.npz: angle_below_offset=10, angle_above_offset=50
# 120506.npz: angle_below_offset=10, angle_above_offset=50
# 121855.npz: angle_below_offset=55, angle_above_offset=5

for index, image in enumerate(intensity_data):
    offset1, offset2 = optimize_offset_2filters(image, offset1, threshold=-1, 
                                                initial=initial, fix_snapback=True, angle_below_offset=55, angle_above_offset=5)
    print(f'{(index + 1) * 10}s: offset 1 = {np.rad2deg(offset1.item())}, offset 2 = {np.rad2deg(offset2.item())}')
    initial = False
    offset_list1.append(np.rad2deg(offset1.item())), offset_list2.append(np.rad2deg(offset2.item()))

# Plot offset angles
time = np.array(range(len(offset_list1))) * 10 + 10
print(offset_list1)
plt.plot(offset_list1, label="offset1")
plt.legend()
plt.show()
plt.plot(offset_list1, label="offset1")
plt.plot(offset_list2, label="offset2")
plt.legend()
plt.show()

# Save model data
offset_list1=np.array(offset_list1)
# file_path = r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\\'
file_path = rf'/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/Temp_Sweep_data/angles/'
full_path = file_path + gif
np.savez(full_path, gif, offset=offset_list1, time=time)
